## MPM Microscopy Data Processing ##

Use three images of each region of interest to create a processed version
 - Take the average of the two TPEF images, use as red channel
 - Use SHG image as blue channel
 - Create a greyscale version from this
 - Save both multi-channel and greyscale images


In [1]:
import numpy as np
import matplotlib.pyplot as plt #requires Pillow to read tiffs
from os import walk #to get directory listing

from skimage.color import rgb2gray
from scipy import ndimage

In [2]:
data_dir = '../py_alpha_amd_release/data/'
subdirs = ['Dysplastic'] #, 'Malignant', 'Healthy']
path = data_dir + '%s/%s/med/' #Substitute in subdirectory and slide ID to get full path
#path = data_dir + '%s/%s/' #Substitute in subdirectory and slide ID to get full path
#file_name = '%s_%s-p%d_ch%d' #filename consists of slideid, region id, polarization, channel

out_file = '../data/processed/%s_%s.tif' #substitute in slide ID and region ID
out_file_greyscale = '../data/processed/%s_%s_gs.tif' #substitute in slide ID and region ID

In [9]:
#For each of the data subdirectories get a list of slides (each is a subdirectory of that)
#Then iterate the slides and process each region.
for sd in subdirs:
    (_, slidedirs, _) = next(walk(data_dir+sd))
    
    for slide in slidedirs:
        print('Processing slide', slide, 'in', sd)
        image_folder = path % (sd, slide) + '/'
        (_, _, images) = next(walk(image_folder))
        
        #Sort alphabetically to get the files for one region together. Then iterate through
        #regions and create the processed images
        images = sorted(images)
        numRegions = len(images)//3
        assert(numRegions*3 == len(images))
        
        print('Found %d regions'%(numRegions,))
        for roi in range(numRegions):
            roi_name = images[3*roi].split('-')[0]
            shg = plt.imread(image_folder+images[3*roi])/255.
            blue_ch = np.reshape(shg, (*shg.shape,1))
            
            tpef1 = plt.imread(image_folder+images[3*roi + 1])
            tpef2 = plt.imread(image_folder+images[3*roi + 2])
            red_ch = ((tpef1 + tpef2) / 2)/255.
            red_ch = np.reshape(red_ch, (*red_ch.shape,1))

            processed = np.append(red_ch, np.zeros(red_ch.shape, dtype=int), axis=-1)
            processed = np.append(processed, blue_ch, axis=-1)
            
            #MPM images are rotated by 90 degrees counter clockwise compared to brightfield. Rotate back
            processed = ndimage.rotate(processed, -90)
            #handle rounding errors which lead to sub-zero intensities
            processed = np.max(processed, 0)
            
            plt.imsave(out_file%(slide,roi_name), processed)
            plt.imsave(out_file_greyscale%(slide,roi_name), rgb2gray(processed), cmap='gray', vmin=0, vmax=1)


Processing slide 148185 in Dysplastic
Found 20 regions
(512, 3) 0.0 0.6705882352941176
(512, 3) 0.0 0.8627450980392092
(512, 3) 0.0 0.592156862745098
(512, 3) 0.0 0.7215686274509805
(512, 3) 0.0 0.6980392156862745
(512, 3) 0.0 0.454901960784312
(512, 3) 0.0 0.7568627450980394
(512, 3) 0.0 0.9490196078431437
(512, 3) 0.0 0.8745098039215689
(512, 3) 0.0 0.6588235294117648
(512, 3) 0.0 0.8784313725490199
(512, 3) 0.0 0.7058823529411765
(512, 3) 0.0 0.8313725490196081
(512, 3) 0.0 0.8666666666666667
(512, 3) 0.0 0.7176470588235295
(512, 3) 0.0 0.5000000000000002
(512, 3) 0.0 0.6784313725490196
(512, 3) 0.0 0.596078431372549
(512, 3) 0.0 0.48823529411764716
(512, 3) 0.0 0.7294117647058822
Processing slide 148184 in Dysplastic
Found 7 regions
(512, 3) 0.0 0.5686274509803971
(512, 3) 0.0 0.4588235294117647
(512, 3) 0.0 0.46274509803921604
(512, 3) 0.0 0.5000000000000108
(512, 3) 0.0 0.5960784313725569
(512, 3) 0.0 0.6431372549019667
(512, 3) 0.0 1.0000000000000004


In [8]:
np.max(processed, 0)

array([[0.48627451, 0.        , 0.91372549],
       [0.44313725, 0.        , 0.8       ],
       [0.48039216, 0.        , 0.8627451 ],
       ...,
       [0.47647059, 0.        , 0.76078431],
       [0.48627451, 0.        , 0.81960784],
       [0.49803922, 0.        , 0.77647059]])